# 03 Deploy to Amazon Bedrock

## Amazon Bedrock에 모델 불러오기

### Step 1 

![step1](../images/deploy_1.png)

- Amazon Bedrock 콘솔에서 `Imported models`를 클릭합니다.

### Step 2 

![step2](../images/deploy_2.png)

- `Imported model`을 클릭하여 배포 작업을 시작합니다.

### Step 3

![step3](../images/deploy_3.png)

- `Model name`을 입력하고, 파인 튜닝한 모델을 저장한 S3 location을 선택한 후 `Import model`을 클릭합니다.
- S3 경로는 `{S3버킷}/{training job 이름}/output/model/ `로 설정해야합니다.

### Step 4

![step4](../images/deploy_4.png)

- Model import job은 llama-8b 기준 약 15분이 소요됩니다.
- Model import job이 완료되면 `Models`에서 배포된 모델을 확인할 수 있습니다.

### Step 5

![step5](../images/deploy_5.png)

- 배포가 완료되면 `Bedrock Playground`에서 모델을 테스트해볼 수 있습니다. 
- 아래 sample question으로 시작해보세요.

    ```
    <s>[INST]<<SYS>>
    
    You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_11 (tournament VARCHAR)
    <</SYS>>

    [INST]
    Human:
    Return the SQL query that answers the following question: 1987년에 A가 있었던 대회는 어떤 대회입니까?

    [/INST]
    Assistant:
    ```

## Amazon Bedrock API로 응답 확인하기

- Amazon Bedrock의 invoke API로 llama3 모델을 불러옵니다.

In [ ]:
import boto3
import sagemaker
import json

sess = sagemaker.Session()

region = sess.boto_region_name
client = boto3.client("bedrock-runtime", region_name=region)
model_id = "<ENTER_YOUR_MODEL_ARN_HERE>"

assert model_id != "<ENTER_YOUR_MODEL_ARN_HERE>", "ERROR: Please enter your model id"

def get_sql_query(system_prompt, user_question):
    """
    Generate a SQL query using Llama 3 8B
    Remember to use the same template used in fine tuning
    """
    formatted_prompt = f"<s>[INST] <<SYS>>{system_prompt}<</SYS>>\n\n[INST]Human: Return the SQL query that answers the following question: {user_question}[/INST]\n\nAssistant:"
    native_request = {
        "prompt": formatted_prompt,
        "max_tokens": 100,
        "top_p": 0.9,
        "temperature": 0.1
    }
    response = client.invoke_model(modelId=model_id,
                                   body=json.dumps(native_request))
    response_text = json.loads(response.get('body').read())["outputs"][0]["text"]

    return response_text

### Test set에 있는 질문 실행해보기

In [ ]:
import json

# 데이터셋을 저장할 리스트
dataset = []

# JSON 파일에서 데이터 읽기 (이 부분은 실제 데이터 로딩 방식에 따라 조정 필요)
with open('../datasets/ko_test_dataset.json', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        dataset.append(data['messages'])

# 무작위로 10개의 예제 선택 (실제 환경에서는 random.sample 등을 사용할 수 있습니다)
sample_dataset = dataset[:10]

# 각 데이터셋 항목에 대해 쿼리 생성 및 결과 비교
for item in sample_dataset:
    system_prompt = item[0]['content']
    user_question = item[1]['content'].split(": ", 1)[1]  # "Return the SQL query that answers the following question: " 부분 제거
    assistant_answer = item[2]['content']
    
    # 모델을 사용하여 SQL 쿼리 생성
    generated_query = get_sql_query(system_prompt, user_question)
    
    print(f"질문: {user_question}\n")
    print(f"답변: {generated_query}\n")
    print(f"정답: {assistant_answer}\n")
    print("\n" + "="*50 + "\n")

- `user_question`에 한국어 질문을 직접 작성하여 실행해보세요.

In [ ]:
system_prompt = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You can use the following table schema for context: CREATE TABLE table_name_11 (tournament VARCHAR)"
user_question = "1987년에 A가 있었던 대회는 어떤 대회입니까?"

query = get_sql_query(system_prompt, user_question).strip()
print(query)